# Machine Learning on PGA Tour - Data Collection

In this notebook we will collect the data, handle the potential missing values, and label it. The data is collected with web scraping methods from the official website of PGA Tour: https://www.pgatour.com.

In [36]:
# IMPORTS
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

After importing the needed libraries, we will create the dataframe where we will store the data. First we create the dataframe with default columns: the name of the player and his final ranking in PGA Tour 2021.

In [37]:
# CREATE DATAFRAME FOR THE DATA
df = pd.DataFrame(columns=["Name", "Ranking"])

Now, first we want to find out the rankings of the players and their names during season 2021. To find this data we will use PGA Tour's official site.

In [38]:
# GET DATA OF NAMES
URL = "https://www.pgatour.com/content/pgatour/stats/stat.02671.y2021.html"
response = requests.get(URL)
content = response.content
soup = BeautifulSoup(content, "html.parser")
table = soup.find(id="statsTable")
rows = table.find_all("tr")
names = []
for row in rows[1:126]:
    name = row.find("a").text.strip()
    ranking = int(row.find("td").text.strip())
    names.append((ranking, name))
names = np.array(names)
df["Name"] = names[:, 1]
df["Ranking"] = names[:, 0]

Then we will collect the values for the features. To do this, we will first create a function that will search the values from a given URL and add these values to the dataframe.

In [39]:
def scrape_data(url, header, df):
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    table = soup.find(id="statsTable")
    rows = table.find_all("tr")
    values = []
    for row in rows[1:]:
        name = row.find("a").text.strip()
        value = float(row.find_all("td")[4].text.strip().replace(",", ""))
        values.append((name, value))
    df2 = pd.DataFrame(values, columns=["Name", header])
    result = pd.merge(left=df, right=df2, on="Name", how="outer").iloc[:125]
    return result

Then we will find out the average driving distances of the players (in yards) in the regular season.

In [40]:
# GET DATA OF DRIVING DISTANCE
URL = "https://www.pgatour.com/content/pgatour/stats/stat.101.y2021.eoff.t013.html"
header = "Driving Distance"
df = scrape_data(URL, header, df)
df

,Name,Ranking,Driving Distance
0,Patrick Cantlay,1,301.7
1,Jon Rahm,2,307.7
2,Kevin Na,3,288.1
3,Justin Thomas,4,302.7
4,Viktor Hovland,5,300.5
...,...,...,...
120,C.T. Pan,121,296.3
121,Matt Kuchar,122,288.0
122,Brice Garnett,123,288.1
123,Scott Stallings,124,298.2


Now our dataframe has the name of the player, ranking, and the average driving distance of the player during the regular season of PGA Tour 2021. Let's see do we have some null values at this point:

In [41]:
df[df["Driving Distance"].isnull()]

,Name,Ranking,Driving Distance
106,Garrick Higgo,107,NaN


In our data there is one player who does not have a driving distance in the data source. There are multiple ways how we can fix this problem. As it is only one player, we choose to manually fill the driving distance. According to [PGA Tour statistics](https://www.pgatour.com/players/player.54421.garrick-higgo.html), Garrick Higgo's average driving distance during PGA Tour 2021 was 308.2 yards.

In [42]:
df.at[106, "Driving Distance"] = 308.2
df[df["Name"] == "Garrick Higgo"]

,Name,Ranking,Driving Distance
106,Garrick Higgo,107,308.2


In [43]:
df[df["Driving Distance"].isnull()]

,Name,Ranking,Driving Distance


Now all of our players have a driving distance. Let's continue to add other features to our dataset. The next one is Driving Accuracy, which tells the percentage of time a tee shot comes to rest in the fairway. This feature is quite important as it doesn't matter how long a tee shot is if it ends up in the forest.

In [44]:
# GET DATA OF DRIVING DISTANCE
URL = "https://www.pgatour.com/content/pgatour/stats/stat.102.y2021.eoff.t013.html"
header = "Driving Accuracy"
df = scrape_data(URL, header, df)
df

,Name,Ranking,Driving Distance,Driving Accuracy
0,Patrick Cantlay,1,301.7,60.28
1,Jon Rahm,2,307.7,63.52
2,Kevin Na,3,288.1,66.73
3,Justin Thomas,4,302.7,55.68
4,Viktor Hovland,5,300.5,62.47
...,...,...,...,...
120,C.T. Pan,121,296.3,61.03
121,Matt Kuchar,122,288.0,65.81
122,Brice Garnett,123,288.1,70.86
123,Scott Stallings,124,298.2,58.83


Now let's see if we have some null values.

In [45]:
df[df["Driving Accuracy"].isnull()]

,Name,Ranking,Driving Distance,Driving Accuracy
106,Garrick Higgo,107,308.2,NaN


Same player again. There seems to be something with the data of Higgo. Let's manually fill this value again.  

In [46]:
df.at[106, "Driving Accuracy"] = 55.71
df[df["Name"] == "Garrick Higgo"]

,Name,Ranking,Driving Distance,Driving Accuracy
106,Garrick Higgo,107,308.2,55.71


Let's proceed to the next feature, which is the Club Head Speed. This value tells the speed at which the club impacts the ball on Par 4 and Par 5 tee shots. The value is in miles per hour (mph).

In [47]:
# GET DATA OF DRIVING DISTANCE
URL = "https://www.pgatour.com/content/pgatour/stats/stat.02401.y2021.eoff.t013.html"
header = "Club Head Speed"
df = scrape_data(URL, header, df)
df

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed
0,Patrick Cantlay,1,301.7,60.28,116.03
1,Jon Rahm,2,307.7,63.52,118.23
2,Kevin Na,3,288.1,66.73,112.39
3,Justin Thomas,4,302.7,55.68,117.08
4,Viktor Hovland,5,300.5,62.47,116.50
...,...,...,...,...,...
120,C.T. Pan,121,296.3,61.03,111.20
121,Matt Kuchar,122,288.0,65.81,108.60
122,Brice Garnett,123,288.1,70.86,109.53
123,Scott Stallings,124,298.2,58.83,115.96


I have a feeling we have one missing value. Let's see.

In [48]:
df[df["Club Head Speed"].isnull()]

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed
106,Garrick Higgo,107,308.2,55.71,NaN


"How did you do that?". OK, let's fill this value manually again.

In [49]:
df.at[106, "Club Head Speed"] = 118.77
df[df["Name"] == "Garrick Higgo"]

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed
106,Garrick Higgo,107,308.2,55.71,118.77


Now for our next feature, which is the Ball Speed. This value tells the peak speed of the golf ball at launch on Par 4 and Par 5 tee shots. The unit is again miles per hour (mph).

In [50]:
URL = "https://www.pgatour.com/content/pgatour/stats/stat.02402.y2021.eoff.t013.html"
header = "Ball Speed"
df = scrape_data(URL, header, df)
df

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed,Ball Speed
0,Patrick Cantlay,1,301.7,60.28,116.03,174.53
1,Jon Rahm,2,307.7,63.52,118.23,177.44
2,Kevin Na,3,288.1,66.73,112.39,164.79
3,Justin Thomas,4,302.7,55.68,117.08,176.24
4,Viktor Hovland,5,300.5,62.47,116.50,173.60
...,...,...,...,...,...,...
120,C.T. Pan,121,296.3,61.03,111.20,167.34
121,Matt Kuchar,122,288.0,65.81,108.60,162.18
122,Brice Garnett,123,288.1,70.86,109.53,164.71
123,Scott Stallings,124,298.2,58.83,115.96,173.80


Again let's see if Higgo's value is missing.

In [51]:
df[df["Ball Speed"].isnull()]

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed,Ball Speed
106,Garrick Higgo,107,308.2,55.71,118.77,NaN


This time, PGA official website doesn't have any statistics for Higgo's ball speed. Therefore, to obtain this value we have to do something else. One way to obtain this value is to fill it with the average of the values around it. This way we don't have to lose information and we get an approximate value that should not appear as an outlier.

In [52]:
df.at[106, "Ball Speed"] = round(df.iloc[96:116, 5].mean(), 2)
df[df["Name"] == "Garrick Higgo"]

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed,Ball Speed
106,Garrick Higgo,107,308.2,55.71,118.77,169.89


Now for our last feature, which is the Spin Rate. This feature tells the spin rate of the ball in RPM's immediately after leaving the club on Par 4 and Par 5 tee shots.

In [53]:
# GET DATA OF DRIVING DISTANCE
URL = "https://www.pgatour.com/content/pgatour/stats/stat.02405.y2021.eoff.t013.html"
header = "Spin Rate"
df = scrape_data(URL, header, df)
df

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed,Ball Speed,Spin Rate
0,Patrick Cantlay,1,301.7,60.28,116.03,174.53,2397.4
1,Jon Rahm,2,307.7,63.52,118.23,177.44,2348.7
2,Kevin Na,3,288.1,66.73,112.39,164.79,2805.7
3,Justin Thomas,4,302.7,55.68,117.08,176.24,2281.8
4,Viktor Hovland,5,300.5,62.47,116.50,173.60,2389.4
...,...,...,...,...,...,...,...
120,C.T. Pan,121,296.3,61.03,111.20,167.34,2129.2
121,Matt Kuchar,122,288.0,65.81,108.60,162.18,2419.4
122,Brice Garnett,123,288.1,70.86,109.53,164.71,2539.5
123,Scott Stallings,124,298.2,58.83,115.96,173.80,2516.0


Let's fill Higgo's Spin Rate the same way we did his Ball Speed.

In [60]:
df.at[106, "Spin Rate"] = round(df.iloc[96:116, 6].mean(), 2)
df[df["Name"] == "Garrick Higgo"]

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed,Ball Speed,Spin Rate,Eligible
106,Garrick Higgo,107,308.2,55.71,118.77,169.89,2489.11,-1.0


Now we have all of our features collected. Next we have to label the data. As only the top 30 players get to participate in the Tour Championship, the final stage of the FedEx Cup playoffs, we can label our data with 1 if the player's ranking is 30 or smaller and with -1 if the player's ranking is over 30.

In [61]:
df.loc[df["Ranking"].astype(int) <= 30, "Eligible"] = 1
df.loc[df["Ranking"].astype(int) > 30, "Eligible"] = -1
df

,Name,Ranking,Driving Distance,Driving Accuracy,Club Head Speed,Ball Speed,Spin Rate,Eligible
0,Patrick Cantlay,1,301.7,60.28,116.03,174.53,2397.4,1.0
1,Jon Rahm,2,307.7,63.52,118.23,177.44,2348.7,1.0
2,Kevin Na,3,288.1,66.73,112.39,164.79,2805.7,1.0
3,Justin Thomas,4,302.7,55.68,117.08,176.24,2281.8,1.0
4,Viktor Hovland,5,300.5,62.47,116.50,173.60,2389.4,1.0
...,...,...,...,...,...,...,...,...
120,C.T. Pan,121,296.3,61.03,111.20,167.34,2129.2,-1.0
121,Matt Kuchar,122,288.0,65.81,108.60,162.18,2419.4,-1.0
122,Brice Garnett,123,288.1,70.86,109.53,164.71,2539.5,-1.0
123,Scott Stallings,124,298.2,58.83,115.96,173.80,2516.0,-1.0


Now our dataset is ready. Let's save the data into a CSV-file and continue solving this Machine Learning in another notebook.

In [62]:
df.to_csv("../data/pga_data.csv", index=False)